In [9]:
from dotenv import load_dotenv

load_dotenv("../.envrc")


True

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 1000자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))
retriever = vectorstore.as_retriever()

In [11]:
#GPT 3.5 모델 선언
from langchain import hub
import os
llm = ChatOpenAI(model="gpt-4o-mini")

#Langchain Hub에서 RAG 프롬프트 호출
prompt = hub.pull("rlm/rag-prompt", api_key=os.environ["LANGSMITH_API_KEY"])

#Retriever로 검색한 유사 문서의 내용을 하나의 string으로 결합
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [13]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [14]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
answer = rag_chain.invoke("국회의원의 의무는 뭐야?")
print(answer)

국회의원의 의무에는 청렴의 의무와 국가이익을 우선하여 양심에 따라 직무를 수행해야 하는 의무가 포함됩니다. 또한, 국회의원은 법률이 정하는 직을 겸할 수 없으며, 직무상 발언과 표결에 대해 국회 외에서 책임을 지지 않습니다. 이 외에도 국회의원의 지위를 남용하여 개인적인 이익을 추구해서는 안 됩니다.


In [16]:
rag_chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

In [17]:
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import BaseChatMessageHistory, RunnableWithMessageHistory

# PDF 파일 로드 및 처리
loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")

# 1,000자씩 분할하여 Document 객체 형태로 docs에 저장
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

# Chroma 벡터 저장소 설정 및 retriever 생성
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(model='text-embedding-3-small'))
retriever = vectorstore.as_retriever()

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the contextualize question prompt
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [19]:
from langchain_core.messages import AIMessage, HumanMessage

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

chat_history = [
    HumanMessage(content='대통령의 임기는 몇년이야?'),
    AIMessage(content='대통령의 임기는 5년입니다.')
]

contextualize_q_prompt.invoke({"input":"국회의원은?", "chat_history" : chat_history})

ChatPromptValue(messages=[SystemMessage(content='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.', additional_kwargs={}, response_metadata={}), HumanMessage(content='대통령의 임기는 몇년이야?', additional_kwargs={}, response_metadata={}), AIMessage(content='대통령의 임기는 5년입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='국회의원은?', additional_kwargs={}, response_metadata={})])

In [20]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
result = history_aware_retriever.invoke({"input":"국회의원은?", "chat_history" : chat_history})
for i in range(len(result)):
    print(f"{i+1}번째 유사 청크")
    print(result[i].page_content[:250])
    print("-"*100)

1번째 유사 청크
법제처                                                            5                                                       국가법령정보센터
대한민국헌법
③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.
 
제42조 국회의원의 임기는 4년으로 한다.
 
제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
 
제44조 ①국회의원은 현
----------------------------------------------------------------------------------------------------
2번째 유사 청크
법제처                                                            5                                                       국가법령정보센터
대한민국헌법
③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.
 
제42조 국회의원의 임기는 4년으로 한다.
 
제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
 
제44조 ①국회의원은 현
----------------------------------------------------------------------------------------------------
3번째 유사 청크
법제처                                                            5                                                       국가법령정보센터
대한민국헌법
③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.
 
제42조 국회의원의 임기는 4년으로 한다.
 
제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.
 
제44조 ①국회의원은 현
---------------

In [21]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [24]:
from langchain_core.messages import HumanMessage
#채팅 히스토리를 적재하기 위한 리스트
chat_history = []

question = "대통령의 임기는 몇년이야?"
#첫 질문에 답변하기 위한 rag_chain 실행
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
#첫 질문과 답변을 채팅 히스토리로 저장
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "국회의원은?"
#두번째 질문 입력 시에는 첫번째 질문-답변이 저장된 chat_history가 삽입됨
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

국회의원의 임기는 4년으로 정해져 있습니다.


In [25]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#채팅 세션별 기록 저장 위한 Dictionary 선언
store = {}

#주어진 session_id 값에 매칭되는 채팅 히스토리 가져오는 함수 선언
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


#RunnableWithMessageHistory 모듈로 rag_chain에 채팅 기록 세션별로 자동 저장 기능 추가
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [26]:
conversational_rag_chain.invoke(
    {"input": "대통령의 임기는 몇년이야?"},
    config={
        "configurable": {"session_id": "240510101"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'대통령의 임기는 5년이며, 중임할 수 없습니다.'

In [27]:
conversational_rag_chain.invoke(
    {"input": "국회의원은?"},
    config={"configurable": {"session_id": "240510101"}},
)["answer"]

'국회의원의 임기는 4년입니다.'

In [28]:
import torch

# PyTorch 버전 확인
print(f"PyTorch 버전: {torch.__version__}")

# CUDA 버전 확인 (CUDA를 사용할 수 있는 경우)
if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
else:
    print("CUDA를 사용할 수 없습니다.")

# CUDA 사용 가능 여부 및 디바이스 설정
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA를 사용할 수 있습니다. 사용 가능한 GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA를 사용할 수 없습니다. CPU를 사용합니다.")

print(f"현재 사용 중인 디바이스: {device}")

PyTorch 버전: 2.7.1
CUDA를 사용할 수 없습니다.
CUDA를 사용할 수 없습니다. CPU를 사용합니다.
현재 사용 중인 디바이스: cpu


In [30]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="EEVE-Korean-10.8B:latest")
prompt = ChatPromptTemplate.from_template("{topic}에 대한 짧은 농담을 들려주세요. ")

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"topic": "우주여행"}))

옛날 옛적에 우주에서 아주 먼 행성인 플루토니아에 사는 외계인 과학자, 프로페서 블라스트가 있었습니다. 그는 우주의 신비를 풀기 위해 평생을 바쳤죠. 어느 날, 그는 태양계를 여행하는 유랑 우주선단을 만났고, 그 중 한 명인 캡틴 코믹이 그에게 재미있는 농담을 들려주었습니다:

"왜 외계인들은 항상 지구를 방문하려고 하는 걸까요? 왜냐하면 우리 행성이 '지구'라고 불리기 때문이죠!"

프로페서 블라스트는 크게 웃었고, 그 후로 그는 우주에서 가장 멋진 농담으로 이 이야기를 기억했습니다.


In [32]:
from langchain_chroma import Chroma

Chroma().delete_collection()

In [37]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_ollama import ChatOllama
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

loader = PyPDFLoader(r"../data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'} # or 'cuda' if you have a GPU
encode_kwargs = {'normalize_embeddings': True}

embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma.from_documents(docs, embedding)
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt", api_key=os.environ["LANGSMITH_API_KEY"])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [38]:
for chunk in rag_chain.stream("헌법 제 1조 1항이 뭐야"):
    print(chunk, end="", flush=True)

헌법 제1조 1항은 대한민국의 기본 원칙과 방향을 명시하고 있습니다. 이 조항에는 대한민국임시정부의 법통 계승, 불의에 맞선 4·19 민주이념의 이어짐, 정의, 인도, 동포애를 바탕으로 한 민족 단결 강화, 사회적 폐습 및 불의 타파, 자유민주적 기본질서 확립, 그리고 정치, 경제, 사회, 문화 등 모든 영역에서의 기회 균등과 개인의 능력 최고도 발휘가 포함됩니다.